A Face image recognizer and classifier that uses the inbuilt OpenCV Face classifier to recognize between 5 most popular actors from the Avengers movie franchise. Uses a haarCascade for detection of a face.

In [1]:
import os
import cv2 as cv
import numpy as np

Getting the Absolute path to the images directory

In [2]:
IMGS_PATH = os.path.join(os.getcwd(), "Images\\train")
actors = ['chris_evans', 'chris_hemsworth', 'mark_ruffalo', 'robert_downey_jr', 'scarlett_johansson']

Building the Facial cascade from its xml file 

In [3]:
FacialHC = cv.CascadeClassifier("haarcascade_face.xml")

A function to obtain the features and class labels of each of the respective images from the training data.

Features include the area of the image that is detected to be a face by the facedetector.

Labels correspond to the indices of the respective actors in the actors list.

In [4]:
def extract_features():
    features = []
    classLabels = []
    for actor in actors:
        dir_path = os.path.join(IMGS_PATH, actor)
        label = actors.index(actor)
        for image in os.listdir(dir_path):
            img_path = os.path.join(dir_path, image)
            img = cv.imread(img_path)
            gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

            face_rectangles = FacialHC.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=4)
            for (x, y, w, h) in face_rectangles:
                regionofInterest = gray_img[x : x+w, y : y+h]
                features.append(regionofInterest)
                classLabels.append(label)
    return (features, classLabels)

**Main** : getting the features, labels and instantiating an OpenCV FACErecognizer instance that trains on these features and labels.

In [5]:
features, labels = extract_features()
features = np.array(features, dtype=object)
labels = np.array(labels)

face_recognizer = cv.face.LBPHFaceRecognizer_create()

face_recognizer.train(features, labels)
print("---------------Training complete----------------")

---------------Training complete----------------


Saving the Model , can also save the features and labels as .npys (optional)

In [6]:
face_recognizer.save("Trained_faces.yml")

np.save("Facefeatures.npy", features)
np.save("Faceclasses.npy", labels)

**Testing/Validating the model**

In [7]:
face_recognizer.read("Trained_faces.yml")
TEST_PATH = os.path.join(os.getcwd(), "Images\\test")

Selecting random test images from each of the actors

In [8]:
import random

In [9]:
tests = []

for actor in actors:
    actor_test_path = os.path.join(TEST_PATH, actor)
    test_img_name = random.choice(os.listdir(actor_test_path))
    test_img_path = os.path.join(actor_test_path, test_img_name)
    img = cv.imread(test_img_path)
    gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    tests.append(gray_img)

In [12]:
random.shuffle(tests)
for test in tests:
    face_rectangles = FacialHC.detectMultiScale(test, 1.1, 4)
    for (x, y, w, h) in face_rectangles:
        roi = test[x : x+w, y : y+h]
        predicted_label, confidence = face_recognizer.predict(roi)
        avenger = actors[predicted_label]
        # print(f'Identified {avenger} with a confidence val of {confidence}')
        print(f'Recognized {avenger} !')
        remake = cv.cvtColor(test, cv.COLOR_GRAY2BGR)
        cv.rectangle(remake, (x,y), (x + w, y + h), (0,255,0), thickness=1)
        cv.putText(remake, str(actors[predicted_label]), (20,20), cv.FONT_HERSHEY_COMPLEX, 1.0, (0,255,255), thickness=2)
        cv.imshow("Avenger detected", remake)
        cv.waitKey(0)

Recognized chris_hemsworth !
Recognized chris_evans !
Recognized mark_ruffalo !
Recognized robert_downey_jr !
Recognized scarlett_johansson !
